### Este es un cuaderno simple para construir, visualizar y diagnosticar el desempeño de los algoritmos AD en el conjunto de datos de planetas habitables (más grande).

Acompaña al Capítulo 3 del libro.

Los datos para este ejercicio provienen de [aquí](https://phl.upr.edu/).

Autora: Viviana Acquaviva, con contribuciones de Jake Postiglione y Olga Privman.

In [ ]:
import pandas as pd

import numpy as np

import sklearn.tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.model_selection import KFold, StratifiedKFold

from scipy import stats

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
from io import StringIO  
from IPython.display import Image  
import pydotplus
from sklearn.tree import export_graphviz

In [ ]:
import matplotlib
font = {'size'   : 20}

matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
pwd

### Paso 1: Análisis/exploración preliminar de datos.

Una vez que estamos trabajando con conjuntos de datos de nivel de investigación, nuestro primer paso siempre debe ser la exploración de datos.

Podemos leer los datos en un marco de datos, como hicimos anteriormente, y hacer un análisis de datos preliminar.

In [ ]:
df = pd.read_csv('../data/phl_exoplanet_catalog.csv', sep = ',')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.groupby('P_HABITABLE').count()

#### Comencemos por agrupar los planetas probable y posiblemente habitables.

In [ ]:
# ¿Qué estamos haciendo aquí? Creando un nuevo marco de datos llamado bindf y eliminando la etiqueta de habitabilidad anterior
bindf = df.drop('P_HABITABLE', axis = 1) 

In [ ]:
# ¿Qué tal aquí? creando nuestra nueva etiqueta de habitabilidad
bindf['P_HABITABLE'] = (np.logical_or((df.P_HABITABLE == 1) , (df.P_HABITABLE == 2))) 

# ¿y aquí? Re-elaboración de esta columna como entero
bindf['P_HABITABLE'] = bindf['P_HABITABLE'].astype(int) 

In [ ]:
bindf.head()

### Seleccionemos algunas columnas.

S_MAG - magnitud de la estrella

S_DISTANCE - distancia a la estrella (parsecs)

S_METALLICITY - metalicidad de la estrella (dex)

S_MASS - masa de la estrella (unidades solares)

S_RADIUS - radio de la estrella (unidades solares)

S_AGE - edad de la estrella (Gaño)

S_TEMPERATURE - temperatura efectiva de la estrella (K)

S_LOG_G - log(g) de la estrella

P_DISTANCE - distancia promedio del planeta a la estrella (AU) 

P_FLUX - flujo eslar promedio del planeta (unidades terrestres)

P_PERIOD - periodo del planeta (días) 

### Podemos seleccionar las mismas características que hicimos en el Capítulo 2.

In [ ]:
final_features = bindf[['S_MASS', 'P_PERIOD', 'P_DISTANCE']] 

In [ ]:
targets = bindf.P_HABITABLE

In [ ]:
final_features.head()

### Hay algunos NaN (del inglés *Not A Number*). Podemos ver esto usando la propiedad "describe", que solo cuenta valores numéricos en cada columna.

In [ ]:
final_features.shape

In [ ]:
final_features.describe()

### Podemos contar los datos faltantes por columna...

In [ ]:
for i in range(final_features.shape[1]):
    print(len(np.where(final_features.iloc[:,i].isna())[0]))

### ...y deshacerse de ellos (Nota: ¡hay estrategias de imputación mucho mejores!)

In [ ]:
final_features = final_features.dropna(axis = 0) # se deshace de cualquier instancia con al menos un NaN en cualquier columna
final_features.shape

### Registro de Apredizaje

P: ¿Qué es un 'NaN' y por qué necesitamos eliminarlo de estos datos?

<details>
<summary style="display: list-item;"> ¡Haz click aquí para la respuesta!</summary>
<p>
NaN significa "No es un Número" (del inglés Not a Number) y es la forma en que Python nos dice que hay un valor desconocido donde debería haber uno.

Si no los eliminamos de nuestro conjunto de datos, tendremos problemas si intentamos ejecutar cálculos que fallan cuando operamos con valores NaN.

¡Inténtalo! Comente la primera línea del bloque de código anterior y vuelva a ejecutar los siguientes bloques. ¿Algo se ve diferente?
</p>
</details>

### Próximo paso: buscar valores atípicos

**Método** 1 - ¡gráfica!

In [ ]:
plt.hist(final_features.iloc[:,0], bins = 100, alpha = 0.5)

# Hay un valor atípico notable; lo mismo sucede con otras características.

Pero también podríamos haberlo sabido por la diferencia entre la media y la mediana (que, de hecho, es aún más pronunciada para la distancia orbital y el periodo).

In [ ]:
final_features.describe()

In [ ]:
final_features = final_features[(np.abs(stats.zscore(final_features)) < 5).all(axis=1)] 

# Esto elimina valores atípicos > 5 sigma; sin embargo, cuenta desde la media, por lo que podría no ser ideal

In [ ]:
targets = targets[final_features.index]

### Ahora reinicia el índice.

In [ ]:
final_features = final_features.reset_index(drop=True)

In [ ]:
final_features.head()

### Y no olvides hacer lo mismo para el vector etiqueta

In [ ]:
targets = targets.reset_index(drop=True)

In [ ]:
targets.head()

### Comparando las dimensiones, podemos ver que se eliminaron 9 valores atípicos.

In [ ]:
targets.shape

### Revisar el balance del conjunto de datos

In [ ]:
#Manera simple: contar 0/1s, obtener fracción del total

In [ ]:
np.sum(targets)/len(targets)

In [ ]:
np.bincount(targets) #esto muestra la distribución de las dos clases

### Esto nos dice que nuestro conjunto de datos está extremadamente desequilibrado y, por lo tanto, debemos tener cuidado.

#### También podemos echar un vistazo a las dos primeras características, usando diferentes símbolos para las dos clases.

In [ ]:
plt.figure(figsize=(10,6))

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['#20B2AA','#FF00FF'])

color=cmap(targets)

a = plt.scatter(final_features['S_MASS'], final_features['P_PERIOD'], marker="$\u25EF$",\
                label = 'Prueba', c=targets, cmap=cmap, s=100)

plt.legend()

plt.yscale('log')
plt.xlabel('Masa del Estrella Anfitriona (Unidades de Masa Solar)')
plt.ylabel('Periodo de la Órbita (días)');

bluepatch = mpatches.Patch(color='#20B2AA', label='No Habitable')
magentapatch = mpatches.Patch(color='#FF00FF', label='Habitable')

ax = plt.gca()
leg = ax.get_legend()

plt.legend(handles=[magentapatch, bluepatch],\
           loc = 'lower right', fontsize = 14);

### Registro de Aprendizaje

P: Según este gráfico, ¿esperaría que AD o kVC funcionaran mejor? ¿Por qué?

<details>
<summary style="display: list-item;">¡Haz click aquí para la respuesta!</summary>
<p>
Posiblemente kVC, porque AD solo haría divisiones a lo largo de las características y no puede cortar el conjunto de datos en diagonal.
</p>
</details>

<br/>


P: ¿Qué tipo de desempeño podemos esperar (cualitativamente, es suficiente la información?) ¿Espera tener variables latentes (ocultas) que puedan afectar el resultado más allá de las que tenemos?

<details>
<summary style="display: list-item;">¡Haz click aquí para la respuesta!</summary>
<p>
Hay mucha superposición entre las dos clases, lo que sugiere que no podemos esperar un gran rendimiento a menos que recopilemos más características.</p>
</details>

### Está bien, esto es todo para la exploración preliminar de datos. Hora de implementar.

Comenzamos con una división aleatoria de entrenamiento/prueba, y luego haremos una validación cruzada.

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(final_features, targets, random_state=2)

In [ ]:
Xtrain.shape, Xtest.shape

Elijamos el método AD (fijando el estado aleatorio) y construyamos el modelo.

In [ ]:
model = DecisionTreeClassifier(random_state=5)

model.fit(Xtrain, ytrain)

#### ¡Visualicemos el gráfico!

In [ ]:
# Recordatorio: las características siempre se permutan aleatoriamente en cada división.
# Por lo tanto, la división mejor encontrada puede variar, incluso con los mismos datos de entrenamiento
# y max_features=n_features (¨features¨ se refiere a características), si la mejora del criterio es idéntica
# para varias divisiones enumeradas durante la búsqueda de la mejor división.
# Para obtener un comportamiento determinista durante el ajuste, se debe fijar random_state (estado aleatorio).

dot_data = StringIO()
export_graphviz(
            model,
            out_file =  dot_data,
            feature_names = ['Stellar Mass (M*)', 'Orbital Period (d)', 'Distance (AU)'],
            class_names = ['Not Habitable','Habitable'],
            filled = True,
rounded = True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
nodes = graph.get_node_list()

for node in nodes:
    if node.get_label():
        values = [int(ii) for ii in node.get_label().split('value = [')[1].split(']')[0].split(',')]
        values = [255 * v / sum(values) for v in values]

        values = [int(255 * v / sum(values)) for v in values]

        if values[0] > values[1]:
            alpha = int(values[0] - values[1])
            alpha = '{:02x}'.format(alpha) #turn into hexadecimal
            color = '#20 B2 AA'+str(alpha)
        else:
            alpha = int(values[1] - values[0])
            alpha = '{:02x}'.format(alpha)
            color = '#FF 00 FF'+str(alpha)
        node.set_fillcolor(color)

#graph.write_png('Graph.png',dpi = 300)

Image(graph.create_png())

### Registro de Aprendizaje

P: ¿Puedes predecir la nota de exactitud en el conjunto de entrenamiento?

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
100%, como todos los árboles de decisión "desatados".
</p>
</details>

### Vamos a echar un vistazo a las notas de entrenamiento y de prueba

In [ ]:
print(metrics.accuracy_score(ytrain, model.predict(Xtrain))) #nota de entrenamiento
print(metrics.accuracy_score(ytest, model.predict(Xtest))) #nota de prueba

Parece muy alta, pero ¿cómo se compara con la exactitud de un clasificador vago (del inglés Lazy Classifier) que pone todo en la categoria "no habitable"?

In [ ]:
# Clasificador inocente (del inglés Dummy classifier)

print(metrics.accuracy_score(ytest, np.zeros(len(ytest)))) #desempeño de un clasificador inocente


### Podemos ver otras métricas.

In [ ]:
print(metrics.precision_score(ytest,model.predict(Xtest)))

In [ ]:
print(metrics.recall_score(ytest,model.predict(Xtest)))

¿Cómo son en general?


No buenas.




### ¿Sabes qué necesitaríamos para entender exactamente cómo el modelo está trabajando? ¡Una matriz de confusión!

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de confusión',
                          cmap=plt.cm.Blues):
    """
    Esta función imprime y grafica la matriz de confusión
    Se puede aplicar la normalización ajustando `normalize=True`
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusión normalizada")
    else:
        print('Matriz de confusión sin normalización')

    plt.figure(figsize=(7,6))
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center", verticalalignment="center",
                 color="green" if i == j else "red", fontsize = 30)

    plt.tight_layout()
    plt.ylabel('Etiqueta verdadera')
    plt.xlabel('Etiqueta predicha')

### Podemos graficar la matriz de confusión

Notar que hasta ahora hemos usado las predicciones de *una* iteración de prueba, por lo que los números se referirán solo al conjunto de prueba.

In [ ]:
cm = metrics.confusion_matrix(ytest,model.predict(Xtest))

plot_confusion_matrix(cm, ['No Hab','Hab'], cmap = plt.cm.Pastel2)

### Podemos implementar ahora tres tipos de Validaciones Cruzadas de k-Iteraciones.

Nota: podemos fijar la semilla aleatoria para reproducir exactamente el mismo comportamiento.

En resumen: usar el segundo y el tercer método.

In [ ]:
# Esta es la versión estándar. Importante: no desordena los datos,
# por lo que si todos los ejemplos positivos están al inicio o al final, 
# esto podría producir resultados desastrosos

cv1 = KFold(n_splits = 5)

#Esta es la versión 2: desorden agregado (¡recomendado!)

cv2 = KFold(shuffle = True, n_splits = 5, random_state=5)

# La ESTRATIFICACIÓN garantiza que las distribuciones de clases en cada división 
# se parezcan a las del conjunto de datos completo

cv3 = StratifiedKFold(shuffle = True, n_splits = 5, random_state=5)


### Efecto de la estratificación: veamos el conteo de clases en cada conjunto de divisiones.

In [ ]:
for train, test in cv1.split(final_features, targets): #Justo como son en el conjunto de datos original
...     print('train -  {}   |   test -  {}'.format(
...         np.bincount(targets.loc[train]), np.bincount(targets.loc[test])))

In [ ]:
for train, test in cv2.split(final_features, targets): #Una selección aleatoria
...     print('train -  {}   |   test -  {}'.format(
...         np.bincount(targets.loc[train]), np.bincount(targets.loc[test])))

In [ ]:
for train, test in cv3.split(final_features, targets): #tomando en cuenta la seleccción aleatoria
...     print('train -  {}   |   test -  {}'.format(
...         np.bincount(targets.loc[train]), np.bincount(targets.loc[test])))

#### La función práctica cross\_validate proporciona las notas (especificadas por el parámetro de notas elegido), en forma de diccionario.

In [ ]:
scores1 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv1, scoring = 'accuracy')

scores2 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv2, scoring = 'accuracy')

scores3 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv3, scoring = 'accuracy')

In [ ]:
scores1

#### Podemos ahora calcular un promedio y una desviación estándar

In [ ]:
print("{:.3f}".format(scores1['test_score'].mean()), "{:.3f}".format(scores1['test_score'].std()))

In [ ]:
print("{:.3f}".format(scores2['test_score'].mean()), "{:.3f}".format(scores2['test_score'].std()))

In [ ]:
print("{:.3f}".format(scores3['test_score'].mean()), "{:.3f}".format(scores3['test_score'].std()))

### Registro de Aprendizaje

P: ¿Son las diferencias estadísticamente significativas?

<details>
<summary style="display: list-item;">¡Haz click aquí para la respuesta!</summary>
<p>
¡No, porque las diferencia es menor que una desviación estándar!
</p>
</details>

### Vamos a usar ahora "recall" (recordar) cómo nuestro parámetro de nota. ¿Cambiará el modelo?

In [ ]:
scores1 = cross_validate(DecisionTreeClassifier(random_state=1), final_features, targets, cv = cv1, scoring = 'recall')

scores2 = cross_validate(DecisionTreeClassifier(random_state=1), final_features, targets, cv = cv2, scoring = 'recall')

scores3 = cross_validate(DecisionTreeClassifier(random_state=1), final_features, targets, cv = cv3, scoring = 'recall')

In [ ]:
print("{:.3f}".format(scores1['test_score'].mean()), "{:.3f}".format(scores1['test_score'].std()))
print("{:.3f}".format(scores2['test_score'].mean()), "{:.3f}".format(scores2['test_score'].std()))
print("{:.3f}".format(scores3['test_score'].mean()), "{:.3f}".format(scores3['test_score'].std()))

### Si lo deseamos, también podemos pedir las notas de entrenamiento. Esto es muy útil cuando se diagnostica sesgo vs varianza.

In [ ]:
scores1 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv1, scoring = 'recall', \
                         return_train_score = True)

scores2 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv2, scoring = 'recall', \
                         return_train_score = True)

scores3 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv3, scoring = 'recall',
                         return_train_score = True)

In [ ]:
print("{:.3f}".format(scores1['test_score'].mean()), "{:.3f}".format(scores1['train_score'].mean()))
print("{:.3f}".format(scores2['test_score'].mean()), "{:.3f}".format(scores2['train_score'].mean()))
print("{:.3f}".format(scores3['test_score'].mean()), "{:.3f}".format(scores3['train_score'].mean()))

### La función cross\_validate es útil para calcular la nota, pero no produce etiquetas previstas.

#### Se pueden obtener usando la función cross\_val\_predict, que guarda las predicciones para cada una de las k-iteraciones de prueba y los compila juntos.

In [ ]:
model1 = DecisionTreeClassifier(random_state = 3)

y1 = cross_val_predict(model1, final_features, targets, cv = cv1) #estas son las predicciones,
                                                                #y son independientes del parámetros nota

Esta salida es útil para construir la matriz de confusión "completa":

In [ ]:
metrics.confusion_matrix(targets,y1)

### Sin embargo, las cosas pueden cambiar si usamos un esquema de validación cruzada diferente:

In [ ]:
model1 = DecisionTreeClassifier(random_state = 3)

y1 = cross_val_predict(model1, final_features, targets, cv = cv1)

In [ ]:
model2 = DecisionTreeClassifier(random_state = 3)

y2 = cross_val_predict(model2, final_features, targets, cv = cv2)

In [ ]:
np.sum(y1-y2)

In [ ]:
np.sum(y1)

In [ ]:
metrics.confusion_matrix(targets,y1)

In [ ]:
metrics.confusion_matrix(targets,y2)

Este es un buen recordatorio de que la matriz de confusión también es solo una posible realización del modelo y está sujeta a fluctuaciones aleatorias al igual que las notas de validación cruzada.

### Finalmente, podemos aprender a graficar curvas de aprendizaje utilizando esta función práctica de sklearn.

Las curvas de aprendizaje son útiles para visualizar la nota de entrenamiento vs la nota de prueba, y cómo varían en función del tamaño del conjunto de datos. Nos permiten determinar si tenemos suficientes datos de aprendizaje Y si tenemos un problema de sesgo alto o varianza alta.

El siguiente código fuente es una ligera modificación de [este código](https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html).

In [ ]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=5,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring = 'accuracy'):
    """
    Genera una gráfica simple de la curva de aprendizaje de prueba y entrenamiento.

    Parametros
    ----------
    estimador : tipo de dato "Objecto" que implementa los métodos 'fit y 'predict'
        Un objecto que se clona para cada validación.

    title : Cadena de caracteres
        Título para del gráfico.

    X : vector, forma (n_muestras, n_características)
        Vector de entrenamiento, donde n_muestras es el número de muestras y 
        n_características es el número de características.

    y : vector, forma (n_muestras) o (n_muestras, n_características), opcional
        Objetivo relativo a X para la clasificación o la regresión;
        None para el aprendizaje no supervisado.

    ylim : tupla, forma (ymin, ymax), opcional
        Define los valores mínimos y máximos de y para ser graficados

    cv : entero, Generador de validación cruzada o un iterable, opcional
        Determina la estrategia de división de validación cruzada.
        Las entradas posibles para cv son:
          - Nada (None), para usar la validación cruzada de 3 iteraciones predeterminada,
          - entero, para especificar el número de iteraciones.
          - :term:`CV splitter`,
          - Un iterable que da divisiones (entrenamiento, prueba) como matrices 
          de índices

        Para entradas de entero/None, si ", si el estimador es un clasificador 
        e y es binario o multiclase, se usa :class:`StratifiedKFold`. 
        En todos los demás casos, se utiliza :class:`KFold`.

        Consulte :ref:`User Guide <cross_validation>` para conocer las diversas 
        estrategias de validación cruzada que se pueden utilizar aquí.

    n_jobs : int or None, opcional (default=None)
    Número de trabajos que se ejecutarán en paralelo. 
    ``None``  significa 1 a menos que esté en 
    un contexto :obj:`joblib.parallel_backend`
    ``-1`` significa usar todos los procesadores. 
    Consulte :term:`Glossary <n_jobs>` (glosario) para obtener más detalles.


    train_sizes : vector, forma (n_marcas,), dtype  flotante o entero
    Número relativo o absoluto de ejemplos de entrenamiento que se 
    han utilizado para generar la curva de aprendizaje, Si el dtype es float, 
    se considera como un fracción del tamaño máximo del conjunto de entrenamiento 
    (que se determina por el método de validación seleccionado)
    , es decir, tiene que estar dentro de (0, 1].De lo contrario, 
    se interpreta como tamaños absolutos de los conjuntos de entrenamiento.
    Notese que para la clasificación, el número de muestras generalmente tiene que
    ser lo suficientemente grande para contener al menos una muestra de cada clase.
    (predeterminado: np.linspace(0.1, 1.0, 5))
    """
    plt.figure(figsize=(10,6))
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Ejemplos de Entrenamiento")
    if(scoring=='recall'):
        plt.ylabel("Exhaustividad")
    else:
        plt.ylabel(str(scoring))

    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring = scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Nota de entrenamiento")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Nota de la validación cruzada")

    plt.legend(loc="best")
    return plt

In [ ]:
model = DecisionTreeClassifier(random_state = 5)

In [ ]:
plot_learning_curve(model, 'Árbol de decisión (parámetros predeterminados)', final_features, targets,  cv = cv3, scoring = 'recall');

### Registro de Apredizaje

R: ¿Cómo va nuestro modelo de árbol de decisión? ¿Sufre de una varianza alta o un sesgo alto?

<details>
<summary style="display: list-item;">¡Haz click aquí para la respuesta!</summary>
<p>
El modelo sufre de una varianza muy alta, como se muestra por la diferencia estadísticamente significativa entre las notas de entrenamiento y de prueba. Más datos ayudarán, ya que la pendiente de las notas de prueba parece estar aumentando.
</p>
</details>

#### No veremos clasificadores @ kNN, pero somos (¡por supuesto!) libre de jugar con eso.

El Capítulo 3 del libro analiza aplicaciones adicionales, como los resultados del algoritmo kNN y el caso de un clasificador de 3 clases.


We won't look @ kNN classifier, but you are (of course!) free to play with it.

Chapter 3 of the book discusses additional applications, like the kNN algorithm results, and the case of a 3-class classifier.